In [1]:

# install pakages  (  -> first time )



# 기본 세팅

from urllib.error import HTTPError
import requests
import re

from urllib.request import Request, urlopen
from urllib.parse import quote, urlencode, quote_plus, unquote
from bs4 import BeautifulSoup as bs


import copy
import os
import datetime as dt
import time as t
import numpy as np
import pandas as pd
import random as rd
import warnings
warnings.filterwarnings("ignore")



# 기본 세팅
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotSelectableException
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import InvalidSelectorException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException
# perfomance
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import TouchActions
from selenium.webdriver import ActionChains



from tqdm.notebook import tqdm


In [2]:
from selenium.webdriver.chrome.options import Options

options = Options() 
prefs = {'profile.default_content_setting_values': 
         {'cookies' : 1, 
          'images': 2,
          'javascript' : 1,
          'plugins' : 2, 
          'popups': 2, 
          'geolocation': 2, 
          'notifications' : 2, 
          'auto_select_certificate': 2, 
          'fullscreen' : 2, 
          'mouselock' : 2, 
          'mixed_script': 2, 
          'media_stream' : 2, 
          'media_stream_mic' : 2, 
          'media_stream_camera': 2, 
          'protocol_handlers' : 2, 
          'ppapi_broker' : 2, 
          'automatic_downloads': 2, 
          'midi_sysex' : 2, 
          'push_messaging' : 2, 
          'ssl_cert_decisions': 2, 
          'metro_switch_to_desktop' : 2, 
          'protected_media_identifier': 2, 
          'app_banner': 2, 
          'site_engagement' : 2, 
          'durable_storage' : 2}} 
options.add_experimental_option('prefs', prefs) 
options.add_argument("disable-gpu")
options.add_argument("start-maximized") 
options.add_argument("disable-infobars") 
options.add_argument("--disable-extensions") 
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
#options.add_argument('--headless')

# 조선일보

In [65]:
start_time  = t.time()



for is_data in range(1):
    
    # url 가져오기 (csv파일)
    url_data = pd.read_csv('전체기사 url주소.csv', encoding = 'cp949')
    
    if len(url_data)==0:
        print('기사가 없습니다.')
        continue
    else:
        url_data.columns = ['date', 'urls']

        # 특정 url만 필터링
        url_data = url_data[ url_data['urls'].str.contains('chosun')]

        url_data = url_data.reset_index()
        url_data.columns = ['index', 'date', 'urls']
        urls = list(url_data['urls'])

    # 데이터가 들어갈 공간
    data_set = {}

    #url 접속 후 크롤링
    print("### 본문 수집 ###")
    for idx in tqdm(range(len(urls))):

        # 초기화
        body = ''
        title = ''
        day = ''
        company = ''


        # 조선일보 외 타 신문사 제외
        if ('www.chosun.com' in urls[idx]) or ('biz.chosun' in urls[idx]):
            
            ### parsing ###
            try:
                req = requests.get(urls[idx], headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
                )
                t.sleep(0.1)
                page = req.content
                page = bs(page, 'html.parser')  

            except:
                continue
                
               
            # 페이지를 찾을 수 없습니다.
            if page.h1 != None:
                continue
            
            page_tags = page.find_all()
            body = eval(str(page_tags[0]).split('globalContent=')[1].split('"created_date"')[0].split('"content:"')[0]+'}')
            body["content_elements"]
            temp = []
            for element in body["content_elements"]:
                try:
                    temp.append(element['content'])
                except:
                    pass
            body = '\n'.join(temp)
            
            
            temp = body.split('span')
            temp_list = []
            for string in temp:
                if 'namestock ' in string :
                    continue   
                else:
                    temp_list.append(string)
            if 'span namestock' in body:
                target = body[:]
                del0 = re.search('\nspan namestock.*\dspan', target).group()
                del1 = re.search('span namestock.*\dspan', target).group()
                del2 = re.search('\(.*\dspan', target).group()
                del3 = re.search('\nscript.*jsscript', target).group()
                target = target.replace(del0, '').replace(del1, '').replace(del2, '').replace(del3, '')
                body = target
            else:
                pass
            
            title = page.title.text
            day = url_data['date'][idx]
            company = '조선일보'
            data_set[idx] = [day, title, body, company, urls[idx]]

         

        elif ('it.chosun.com') in urls[idx] :
            urls[idx] = urls[idx].split('?utm_source')[0]
            
            try: 
                req = requests.get(urls[idx], headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
                )
                page = req.content
                page = bs(page, 'html.parser') 
                
            except:
                continue
                
                
            elements = page.find_all('div', 'par')
            body = ''
            for element in elements:
                if element.span != None:
                    continue
                else:
                    body += element.text
            title = page.title.text
            day = url_data['date'][idx]
            company = '조선일보'
            data_set[idx] = [day, title, body, company, urls[idx]]
            
        else:
            continue
            

    '''
    데이터 내보내기 (뉴스본문)
    '''

    
    
    
    
    
    dataframe = pd.DataFrame(data_set).transpose()
    if len(dataframe) ==0:
        print('자료가 없습니다.')
        pass
    else:
        dataframe = pd.DataFrame(data_set).transpose()
        dataframe.columns = ['date', 'title', 'body', 'company', 'url']
        # 본문 내 특수문자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[^\w\s\.\%\)\(\']', repl=r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(pat=r'["\xa0"]', repl = r'', regex=True)
        # 한자 제거
        dataframe['body'] = dataframe['body'].str.replace('\u30fb', ' ')
        dataframe['title'] = dataframe['title'].str.replace('\u30fb', ' ')
        dataframe['body'] = dataframe['body'].str.replace('\xe9', '')
        dataframe['title'] = dataframe['title'].str.replace('\xe9', '')
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u3400-\u9FBF]', repl = r'', regex=True)
        dataframe['body']= dataframe['body'].str.replace(pat=r'["\u2027","\u2026"]', repl = r'', regex=True)
        dataframe['body']= dataframe['body'].str.replace(pat=r'\x29', repl = r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(' b ', '')
        dataframe = dataframe.reset_index()


        for article_idx in range(len(dataframe.body)):
            text = dataframe.body[article_idx][:]
            temp = text.split('span')
            temp_list = []
            for string in temp:
                if 'namestock ' in string :
                    continue   
                else:
                    temp_list.append(string)
            temp = ''.join(temp_list)
            temp = temp.split('script')
            temp = temp[0]
            dataframe.body[article_idx] = temp

        
    dataframe.to_csv('조선일보 크롤링.csv', encoding = "cp949", index= False)




################################################
end_time = t.time()
duration = round((end_time - start_time)/60, 2)


print('작업 완료!')
print('작업 소요시간 : 약 {}분'.format(duration))
print('기사 개수 : ', len(dataframe))
print('작업 완료 시간 : ', str(dt.datetime.now()).split(' ')[0], str(dt.datetime.now()).split(' ')[1].split('.')[0])

### 본문 수집 ###


  0%|          | 0/231 [00:00<?, ?it/s]

작업 완료!
작업 소요시간 : 약 2.94분
기사 개수 :  149
작업 완료 시간 :  2021-09-29 13:51:14


In [25]:
dataframe = pd.DataFrame(data_set).transpose()
dataframe.columns = ['date', 'title', 'body', 'company', 'url']
# 본문 내 특수문자 제거
dataframe['body'] = dataframe['body'].str.replace(pat=r'[^\w\s\.\%\)\(\']', repl=r'', regex=True)
dataframe['body'] = dataframe['body'].str.replace(pat=r'["\xa0"]', repl = r'', regex=True)
# 한자 제거
dataframe['body'] = dataframe['body'].str.replace('\u30fb', ' ')
dataframe['title'] = dataframe['title'].str.replace('\u30fb', ' ')
dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u3400-\u9FBF]', repl = r'', regex=True)
dataframe['body']= dataframe['body'].str.replace(pat=r'["\u2027","\u2026"]', repl = r'', regex=True)
dataframe['body'] = dataframe['body'].str.replace(' b ', '')
dataframe = dataframe.reset_index()
dataframe.to_csv('조선일보 크롤링.csv', encoding = "cp949", index= False)


In [19]:
urls[idx]

'http://it.chosun.com/news/article.html?no=2819343'

In [24]:
req = requests.get(urls[idx], headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
)
page = req.content
page = bs(page, 'html.parser') 
page


<!DOCTYPE html>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<title>NHN엔터테인먼트, 1분기 영업이익 92억원 흑자전환 - IT조선 &gt; 게임·라이프 &gt; 게임·인터넷</title>
<meta content="NHN엔터테인먼트, 1분기 영업이익 92억원 흑자전환" name="title">
<meta content="NHN엔터테인먼트는 2016년 1분기 매출이 전년 같은 기간 대비 47.4% 증가한 2036억원, 영업이익·당기순이익이 각각 92억원과 245억.." name="description"/>
<meta content="IT조선, itchosun, IT조선, 게임·인터넷" name="news_keywords"/>
<meta content="IT조선, IT Chosun, itchosun, IT, 테크, Tech, " name="keywords"/>
<meta content="width=device-width, maximum-scale=1.0" name="viewport">
<meta content="115;56;2016051385005" name="HITLOG"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="http://it.chosun.com/rss/rss.xml" rel="alternate" title="Chosun.com [RSS]" type="application/rss+xml"/>
<!-- dable meta -->
<meta content="http://itimg.chosun.com/sitedata/image/201605/13/2016051385005_0.jpg" property="og:image">
<meta content="article" property="og:type">
<meta content="http://it.chosun.com/site/data/html_dir/2016/

''

In [63]:
dataframe = pd.DataFrame(data_set).transpose()
dataframe.columns = ['date', 'title', 'body', 'company', 'url']
# 본문 내 특수문자 제거
dataframe['body'] = dataframe['body'].str.replace(pat=r'[^\w\s\.\%\)\(\']', repl=r'', regex=True)
dataframe['body'] = dataframe['body'].str.replace(pat=r'["\xa0"]', repl = r'', regex=True)
# 한자 제거
dataframe['body'] = dataframe['body'].str.replace('\u30fb', ' ')
dataframe['title'] = dataframe['title'].str.replace('\u30fb', ' ')
dataframe['body'] = dataframe['body'].str.replace('\xe9', '')
dataframe['title'] = dataframe['title'].str.replace('\xe9', '')
dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u3400-\u9FBF]', repl = r'', regex=True)
dataframe['body']= dataframe['body'].str.replace(pat=r'["\u2027","\u2026"]', repl = r'', regex=True)
dataframe['body']= dataframe['body'].str.replace(pat=r'\x29', repl = r'', regex=True)
dataframe['body'] = dataframe['body'].str.replace(' b ', '')
dataframe = dataframe.reset_index()


dataframe.to_csv('조선일보 크롤링.csv', encoding = "cp949", index= False)


In [61]:
data_set

{0: ['2013.01.04.',
  '[업종돋보기] 대표주 상승에도 어두운 온라인게임株 - 조선비즈',
  '국내 대표 온라인 게임업체인 <span name="stock" id="stock_036570">엔씨소프트(036570)</span>주가가 새해부터 상승세다. 엔씨소프트 주가는 4일 오후 2시 40분 현재 15만9000원을 기록하고 있다. 지난달 5일까지만 해도 14만1000원까지 떨어지며 날개 없는 추락을 보여주던 것에 비하면 한숨을 돌렸다.\n증권사들도 엔씨소프트에 대해 긍정적인 전망을 하고 있다. 지난해 주가가 지나치게 많이 하락했기 때문에 앞으로 오를 여력이 많다는 것이다. 주력 게임들의 해외 매출에 대한 기대감도 작용하고 있다. 성종화 이트레이드증권 연구원은 “최대한 보수적인 관점으로 봐도 엔씨소프트의 올해 영업이익은 작년의 두배가 넘는 3101억원을 기록할 것”이라며 목표주가를 25만원으로 잡았다. 현재 주가에서 60% 정도 오를 수 있는 여력이 있다는 의미다.\n국내 온라인 게임의 대표주인 엔씨소프트는 긍정적인 분위기지만, 다른 온라인 게임업체들의 주가는 요지부동이다. 네오위즈게임즈는 지난달 호재가 연달아 있었지만, 주가는 호재가 나올 때만 반짝 올랐을 뿐이다. 네오위즈게임즈는 지난달 최대 매출원인 크로스파이어 개발사와의 법적 분쟁을 원만하게 합의했고, 윤상규 대표는 박근혜 대통령 당선인의 인수위에 합류하며 주목받았다. 네오위즈게임즈는 호재가 나올 때마다 주가가 상한가를 기록했지만, 4일 현재 주가는 2만4650원으로 지난달 중순 수준에서 정체돼 있다. 작년 1월 4만원대 중반을 기록하던 주가와 비교하면 절반 수준에 그친다.\n<span name="stock" id="stock_181710">NHN(181710)</span>, <span name="stock" id="stock_001040">CJ(001040)</span>E&M, 웹젠, 엠게임##등 다른 온라인 게임 업체들도 주가 흐름은 지지부진하다. 전문가들은 엔씨소프트의 주가가 상승세

In [55]:
dataframe.body[0]

'국내 대표 온라인 게임업체인 주가가 새해부터 상승세다. 엔씨소프트 주가는 4일 오후 2시 40분 현재 15만9000원을 기록하고 있다. 지난달 5일까지만 해도 14만1000원까지 떨어지며 날개 없는 추락을 보여주던 것에 비하면 한숨을 돌렸다.\n증권사들도 엔씨소프트에 대해 긍정적인 전망을 하고 있다. 지난해 주가가 지나치게 많이 하락했기 때문에 앞으로 오를 여력이 많다는 것이다. 주력 게임들의 해외 매출에 대한 기대감도 작용하고 있다. 성종화 이트레이드증권 연구원은 최대한 보수적인 관점으로 봐도 엔씨소프트의 올해 영업이익은 작년의 두배가 넘는 3101억원을 기록할 것이라며 목표주가를 25만원으로 잡았다. 현재 주가에서 60% 정도 오를 수 있는 여력이 있다는 의미다.\n국내 온라인 게임의 대표주인 엔씨소프트는 긍정적인 분위기지만 다른 온라인 게임업체들의 주가는 요지부동이다. 네오위즈게임즈는 지난달 호재가 연달아 있었지만 주가는 호재가 나올 때만 반짝 올랐을 뿐이다. 네오위즈게임즈는 지난달 최대 매출원인 크로스파이어 개발사와의 법적 분쟁을 원만하게 합의했고 윤상규 대표는 박근혜 대통령 당선인의 인수위에 합류하며 주목받았다. 네오위즈게임즈는 호재가 나올 때마다 주가가 상한가를 기록했지만 4일 현재 주가는 2만4650원으로 지난달 중순 수준에서 정체돼 있다. 작년 1월 4만원대 중반을 기록하던 주가와 비교하면 절반 수준에 그친다.\n EM 웹젠 엠게임등 다른 온라인 게임 업체들도 주가 흐름은 지지부진하다. 전문가들은 엔씨소프트의 주가가 상승세를 타는 것과 별개로 온라인 게임 업체들의 전반적인 주가는 올해도 부진할 것으로 전망한다.\n최훈 KB투자증권 연구원은 게임업계의 관심이 모바일로 쏠려 있기 때문에 올해 온라인 게임에서는 이렇다 할 기대작도 나오지 않는다. 대작 온라인 게임인 엑스엘게임즈의 아키에이지가 동시접속자 수 10만명을 기록하며 온라인 게임의 가능성을 확인했지만 다른 게임 업체들의 주가에 긍정적인 영향을 줄 정도는 아니다라며 작년에 주가가 많이 내렸던 

In [33]:
dataframe = pd.DataFrame(data_set).transpose()
dataframe.columns = ['date', 'title', 'body', 'company', 'url']
# 본문 내 특수문자 제거
dataframe['body'] = dataframe['body'].str.replace(pat=r'[^\w\s\.\%\)\(\']', repl=r'', regex=True)
dataframe['body'] = dataframe['body'].str.replace(pat=r'["\xa0"]', repl = r'', regex=True)
# 한자 제거
dataframe['body'] = dataframe['body'].str.replace('\u30fb', ' ')
dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u3400-\u9FBF]', repl = r'', regex=True)
dataframe['body']= dataframe['body'].str.replace(pat=r'["\u2027","\u2026"]', repl = r'', regex=True)
dataframe['body'] = dataframe['body'].str.replace(' b ', '')
dataframe['body'] = dataframe['body'].str.replace('\xe9', '')
dataframe['title'] = dataframe['title'].str.replace('\xe9', '')
dataframe['title'] = dataframe['title'].str.replace('\u30fb', ' ')
dataframe['title'] = dataframe['title'].str.replace(pat=r'[\u3400-\u9FBF]', repl = r'', regex=True)
dataframe['title']= dataframe['title'].str.replace(pat=r'["\u2027","\u2026"]', repl = r'', regex=True)
dataframe['title'] = dataframe['title'].str.replace(' b ', '')
dataframe = dataframe.reset_index()
dataframe
dataframe.to_csv('test.csv', encoding = 'cp949')

In [6]:
dataframe['body']= dataframe['body'].str.replace(pat=r'["\u2027","\u2026", "\xe9"]', repl = r'', regex=True)
dataframe.to_csv('조선일보 크롤링.csv', encoding = "cp949", index= False)


In [20]:
print('\xe9')

é


# 중앙일보

In [ ]:
start_time  = t.time()

for is_data in range(1):
    
    # url 가져오기 (csv파일)
    url_data = pd.read_csv('전체기사 url주소.csv', encoding = 'cp949')
    
    if len(url_data)==0:
        print('기사가 없습니다.')
        continue
    else:
        url_data.columns = ['date', 'urls']

        # 특정 url만 필터링
        url_data = url_data[ url_data['urls'].str.contains('joongang.co.kr')]

        url_data = url_data.reset_index()
        url_data.columns = ['index', 'date', 'urls']
        urls = list(url_data['urls'])

    # 데이터가 들어갈 공간
    data_set = {}



    #url 접속 후 크롤링
    print("### 본문 수집 ###")
    for idx in tqdm(range(len(urls))):

        # 초기화
        body = ''
        title = ''
        day = ''
        company = ''


        # 중앙일보 외 타 신문사 제외
        if 'www.joongang.co.kr' in urls[idx]:


            ### parsing ###
            try:
                req = requests.get(urls[idx], headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
                )
                t.sleep(0.3)
                page = req.content
                page = bs(page, 'html.parser')  

            except:
                continue 
                
            body = page.find_all('div', 'article_body')[0].text
            title = page.title.text
            day = url_data['date'][idx]
            company = '중앙일보'
            data_set[idx] = [day, title, body, company, urls[idx]]
                
            
        else:
            continue
            

    '''
    데이터 내보내기 (뉴스본문)
    '''
    dataframe = pd.DataFrame(data_set).transpose()
    if len(dataframe) ==0:
        print('자료가 없습니다.')
        pass
    else:
        dataframe.columns = ['date', 'title', 'body', 'company', 'url']
        # 본문 내 특수문자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[^\w\s\.\%\)\(\']', repl=r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(pat=r'["\xa0"]', repl = r'', regex=True)
        # 한자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u3400-\u9FBF]', repl = r'', regex=True)
        dataframe['body']= dataframe['body'].str.replace(pat=r'["\u2027","\u2026"]', repl = r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(' b ', '')
        dataframe = dataframe.reset_index()
    dataframe.to_csv('중앙일보 크롤링.csv', encoding = "cp949", index= False)




################################################
end_time = t.time()
duration = round((end_time - start_time)/60, 2)


print('작업 완료!')
print('작업 소요시간 : 약 {}분'.format(duration))
print('기사 개수 : ', len(dataframe))
print('작업 완료 시간 : ', str(dt.datetime.now()).split(' ')[0], str(dt.datetime.now()).split(' ')[1].split('.')[0])



# 동아일보 

In [ ]:
start_time  = t.time()

for is_data in range(1):
    
    # url 가져오기 (csv파일)
    url_data = pd.read_csv('전체기사 url주소.csv', encoding = 'cp949')
    
    if len(url_data)==0:
        print('기사가 없습니다.')
        continue
    else:
        url_data.columns = ['date', 'urls']

        # 특정 url만 필터링
        url_data = url_data[ url_data['urls'].str.contains('donga')]

        url_data = url_data.reset_index()
        url_data.columns = ['index', 'date', 'urls']
        urls = list(url_data['urls'])

    # 데이터가 들어갈 공간
    data_set = {}



    #url 접속 후 크롤링
    print("### 본문 수집 ###")
    for idx in tqdm(range(len(urls))):

        # 초기화
        body = ''
        title = ''
        day = ''
        company = ''


        # 동아일보 외 타 신문사 제외
        if 'www.donga.com' in urls[idx]:


            ### parsing ###
            try:
                req = requests.get(urls[idx], headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
                )
                t.sleep(0.3)
                page = req.content
                page = bs(page, 'html.parser')  

            except:
                continue  

                
                
                
   
            # body type 1
            target = page.find_all('div', 'article_view')[0].find_all('div')
            target = target[0]
            temp = []
            for element in target:
                if element.name == None:
                    temp.append(element)

            body = ' '.join(temp).lstrip().rstrip()                

            
            # body type 2
            if len(body)<=200:
                target = page.find_all('div', 'article_view')[0].find_all('div')
                target = target[0]
                temp = []
                target
                for element in target:
                    if element.name == 'div':
                        for element2 in element:
                            if element2.name == None:
                                temp.append(element2)

                body = ' '.join(temp)
            else:
                pass
                
            
            
            
            
            title = page.title.text
            day = url_data['date'][idx]
            company = '동아일보'
            data_set[idx] = [day, title, body, company, urls[idx]]
                
                
  
        else:
            continue
            

    '''
    데이터 내보내기 (뉴스본문)
    '''
    dataframe = pd.DataFrame(data_set).transpose()
    if len(dataframe) ==0:
        print('자료가 없습니다.')
        pass
    else:
        dataframe.columns = ['date', 'title', 'body', 'company', 'url']
        # 본문 내 특수문자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[^\w\s\.\%\)\(\']', repl=r'', regex=True)
        #dataframe['body'] = dataframe['body'].str.replace(pat=r'["\xa0"]', repl = r'', regex=True)
        # 한자 제거
        #dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u3400-\u9FBF]', repl = r'', regex=True)
        #dataframe['body']= dataframe['body'].str.replace(pat=r'["\u2027","\u2026"]', repl = r'', regex=True)
        #dataframe['body'] = dataframe['body'].str.replace(' b ', '')
        dataframe = dataframe.reset_index()
    dataframe.to_csv('동아일보 크롤링.csv', encoding = "cp949", index= False)





################################################
end_time = t.time()
duration = round((end_time - start_time)/60, 2)


print('작업 완료!')
print('작업 소요시간 : 약 {}분'.format(duration))
print('기사 개수 : ', len(dataframe))
print('작업 완료 시간 : ', str(dt.datetime.now()).split(' ')[0], str(dt.datetime.now()).split(' ')[1].split('.')[0])



In [ ]:
dataframe['title'] = dataframe['title'].str.replace('\u30fb', ' ')
dataframe['title'] = dataframe['title'].str.replace(pat=r'[\u3400-\u9FBF]', repl = r'', regex=True)
dataframe['title']= dataframe['title'].str.replace(pat=r'["\u2027","\u2026"]', repl = r'', regex=True)
dataframe['title']= dataframe['title'].str.replace(pat=r'["\x29"]', repl = r'', regex=True)
dataframe['title'] = dataframe['title'].str.replace(' b ', '')

# 한겨레

In [ ]:
start_time  = t.time()

for is_data in range(1):
    
    # url 가져오기 (csv파일)
    url_data = pd.read_csv('전체기사 url주소.csv', encoding = 'cp949')
    
    if len(url_data)==0:
        print('기사가 없습니다.')
        continue
    else:
        url_data.columns = ['date', 'urls']

        # 특정 url만 필터링
        url_data = url_data[ url_data['urls'].str.contains('hani.co.kr')]

        url_data = url_data.reset_index()
        url_data.columns = ['index', 'date', 'urls']
        urls = list(url_data['urls'])

    # 데이터가 들어갈 공간
    data_set = {}



    #url 접속 후 크롤링
    print("### 본문 수집 ###")
    for idx in tqdm(range(len(urls))):

        # 초기화
        body = ''
        title = ''
        day = ''
        company = ''


        # 한겨레 외 타 신문사 제외
        if 'www.hani.co.kr' in urls[idx]:


            ### parsing ###
            try:
                req = requests.get(urls[idx], headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
                )
                t.sleep(0.3)
                page = req.content
                page = bs(page, 'html.parser')  

            except:
                continue 
                
            body = page.find_all('div', 'article-text')[0].text.rsplit('@hani.co.kr', 1)[0]
            title = page.title.text
            day = url_data['date'][idx]
            company = '한겨레'
            data_set[idx] = [day, title, body, company, urls[idx]]
                
            
        else:
            continue
            

    '''
    데이터 내보내기 (뉴스본문)
    '''
    dataframe = pd.DataFrame(data_set).transpose()
    if len(dataframe) ==0:
        print('자료가 없습니다.')
        pass
    else:
        dataframe.columns = ['date', 'title', 'body', 'company', 'url']
        # 본문 내 특수문자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[^\w\s\.\%\)\(\']', repl=r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(pat=r'["\xa0"]', repl = r'', regex=True)
        # 한자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u3400-\u9FBF]', repl = r'', regex=True)
        dataframe['body']= dataframe['body'].str.replace(pat=r'["\u2027","\u2026"]', repl = r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(' b ', '')
        dataframe = dataframe.reset_index()
    dataframe.to_csv('한겨레 크롤링.csv', encoding = "cp949", index= False)




################################################
end_time = t.time()
duration = round((end_time - start_time)/60, 2)


print('작업 완료!')
print('작업 소요시간 : 약 {}분'.format(duration))
print('기사 개수 : ', len(dataframe))
print('작업 완료 시간 : ', str(dt.datetime.now()).split(' ')[0], str(dt.datetime.now()).split(' ')[1].split('.')[0])



# 경향신문

In [ ]:
start_time  = t.time()

for is_data in range(1):
    
    # url 가져오기 (csv파일)
    url_data = pd.read_csv('전체기사 url주소.csv', encoding = 'cp949')
    
    if len(url_data)==0:
        print('기사가 없습니다.')
        continue
    else:
        url_data.columns = ['date', 'urls']

        # 특정 url만 필터링
        url_data = url_data[ url_data['urls'].str.contains('khan.co.kr')]

        url_data = url_data.reset_index()
        url_data.columns = ['index', 'date', 'urls']
        urls = list(url_data['urls'])

    # 데이터가 들어갈 공간
    data_set = {}



    #url 접속 후 크롤링
    print("### 본문 수집 ###")
    for idx in tqdm(range(len(urls))):

        # 초기화
        body = ''
        title = ''
        day = ''
        company = ''


        # 경향 외 타 신문사 제외
        if 'khan.co.kr' in urls[idx]:


            ### parsing ###
            try:
                req = requests.get(urls[idx], headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
                )
                t.sleep(0.3)
                page = req.content
                page = bs(page, 'html.parser')  

            except:
                continue 
                

            # body = page.find_all('div', 'art_body')[0].text
            body = ""
            for line in page.find_all('p', class_='content_text'):
                body += line.get_text()
            title = page.find_all('title')[0].text
            day = url_data['date'][idx]
            company = '경향'
            data_set[idx] = [day, title, body, company, urls[idx]]

            
        else:
            continue
            

    '''
    데이터 내보내기 (뉴스본문)
    '''
    dataframe = pd.DataFrame(data_set).transpose()
    if len(dataframe) ==0:
        print('자료가 없습니다.')
        pass
    else:
        dataframe.columns = ['date', 'title', 'body', 'company', 'url']
        # 본문 내 특수문자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[^\w\s\.\%\)\(\']', repl=r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(pat=r'["\xa0"]', repl = r'', regex=True)
        # 한자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u3400-\u9FBF]', repl = r'', regex=True)
        dataframe['body']= dataframe['body'].str.replace(pat=r'["\u2027","\u2026"]', repl = r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(' b ', '')
        dataframe = dataframe.reset_index()
    dataframe.to_csv('경향 크롤링.csv', encoding = "cp949", index= False)




################################################
end_time = t.time()
duration = round((end_time - start_time)/60, 2)


print('작업 완료!')
print('작업 소요시간 : 약 {}분'.format(duration))
print('기사 개수 : ', len(dataframe))
print('작업 완료 시간 : ', str(dt.datetime.now()).split(' ')[0], str(dt.datetime.now()).split(' ')[1].split('.')[0])

# 매일경제

In [ ]:
start_time  = t.time()

for is_data in range(1):
    
    # url 가져오기 (csv파일)
    url_data = pd.read_csv('전체기사 url주소.csv', encoding = 'cp949')
    
    if len(url_data)==0:
        print('기사가 없습니다.')
        continue
    else:
        url_data.columns = ['date', 'urls']

        # 특정 url만 필터링
        url_data = url_data[ url_data['urls'].str.contains('mk.co.kr')]

        url_data = url_data.reset_index()
        url_data.columns = ['index', 'date', 'urls']
        urls = list(url_data['urls'])

    # 데이터가 들어갈 공간
    data_set = {}



    #url 접속 후 크롤링
    print("### 본문 수집 ###")
    for idx in tqdm(range(len(urls))):

        # 초기화
        body = ''
        title = ''
        day = ''
        company = ''


        # 매일경제 외 타 신문사 제외
        if 'game' in urls[idx] in urls[idx]:
            continue
            
        elif 'news.mk.co.kr' in urls[idx]:
            ### parsing ###
            try:
                req = requests.get(urls[idx], headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
                )
                t.sleep(0.3)
                page = req.content
                page = bs(page, 'html.parser')  

            except:
                continue  
                

            try:    
                target = page.find_all('div', id='artText')[0]

                temp = []
                for element in target:
                    if element.name == None:
                        if ('=' not in element) & ('div' not in element):
                            if element.lstrip().rstrip() != '':
                                if '//' not in element:
                                    temp.append(element.lstrip().rstrip())
                body = ' '.join(temp)
            except:
                body = page.find_all('div', 'art_txt')[0].text.lstrip().rstrip()
                
            title = page.title.text
            day = url_data['date'][idx]
            company = '매일경제'
            data_set[idx] = [day, title, body, company, urls[idx]]
                
            
        else:
            continue
            

    '''
    데이터 내보내기 (뉴스본문)
    '''
    dataframe = pd.DataFrame(data_set).transpose()
    if len(dataframe) ==0:
        print('자료가 없습니다.')
        pass
    else:
        dataframe.columns = ['date', 'title', 'body', 'company', 'url']
        # 본문 내 특수문자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[^\w\s\.\%\)\(\']', repl=r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(pat=r'["\xa0"]', repl = r'', regex=True)
        # 한자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u3400-\u9FBF]', repl = r'', regex=True)
        dataframe['body']= dataframe['body'].str.replace(pat=r'["\u2027","\u2026"]', repl = r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(' b ', '')
        dataframe = dataframe.reset_index()
    dataframe.to_csv('매일경제 크롤링.csv', encoding = "cp949", index= False)





################################################
end_time = t.time()
duration = round((end_time - start_time)/60, 2)


print('작업 완료!')
print('작업 소요시간 : 약 {}분'.format(duration))
print('기사 개수 : ', len(dataframe))
print('작업 완료 시간 : ', str(dt.datetime.now()).split(' ')[0], str(dt.datetime.now()).split(' ')[1].split('.')[0])

# 한국경제

In [ ]:
start_time  = t.time()

for is_data in range(1):
    
    # url 가져오기 (csv파일)
    url_data = pd.read_csv('전체기사 url주소.csv', encoding = 'cp949')
    
    if len(url_data)==0:
        print('기사가 없습니다.')
        continue
    else:
        url_data.columns = ['date', 'urls']

        # 특정 url만 필터링
        url_data = url_data[ url_data['urls'].str.contains('hankyung')]

        url_data = url_data.reset_index()
        url_data.columns = ['index', 'date', 'urls']
        urls = list(url_data['urls'])

    # 데이터가 들어갈 공간
    data_set = {}



    #url 접속 후 크롤링
    print("### 본문 수집 ###")
    for idx in tqdm(range(len(urls))):

        # 초기화
        body = ''
        title = ''
        day = ''
        company = ''


        # 한국경제 외 타 신문사 제외
        if 'www.hankyung.com' in urls[idx]:


            ### parsing ###
            try:
                req = requests.get(urls[idx], headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
                )
                t.sleep(0.3)
                page = req.content
                page = bs(page, 'html.parser')  

            except:
                continue 
                
            body = page.find('div', id='articletxt').text
            title = page.title.text
            day = url_data['date'][idx]
            company = '한국경제'
            data_set[idx] = [day, title, body, company, urls[idx]]
                
            
        else:
            continue
            

    '''
    데이터 내보내기 (뉴스본문)
    '''
    dataframe = pd.DataFrame(data_set).transpose()
    if len(dataframe) ==0:
        print('자료가 없습니다.')
        pass
    else:
        dataframe.columns = ['date', 'title', 'body', 'company', 'url']
        # 본문 내 특수문자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[^\w\s\.\%\)\(\']', repl=r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(pat=r'["\xa0"]', repl = r'', regex=True)
        # 한자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u3400-\u9FBF]', repl = r'', regex=True)
        dataframe['body']= dataframe['body'].str.replace(pat=r'["\u2027","\u2026"]', repl = r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(' b ', '')
        dataframe = dataframe.reset_index()
    dataframe.to_csv('한국경제 크롤링.csv', encoding = "cp949", index= False)





################################################
end_time = t.time()
duration = round((end_time - start_time)/60, 2)


print('작업 완료!')
print('작업 소요시간 : 약 {}분'.format(duration))
print('기사 개수 : ', len(dataframe))
print('작업 완료 시간 : ', str(dt.datetime.now()).split(' ')[0], str(dt.datetime.now()).split(' ')[1].split('.')[0])

# 디지털타임스

In [ ]:
start_time  = t.time()

for is_data in range(1):
    
    # url 가져오기 (csv파일)
    url_data = pd.read_csv('전체기사 url주소.csv', encoding = 'cp949')
    
    if len(url_data)==0:
        print('기사가 없습니다.')
        continue
    else:
        url_data.columns = ['date', 'urls']

        # 특정 url만 필터링
        url_data = url_data[ url_data['urls'].str.contains('dt.co.kr')]

        url_data = url_data.reset_index()
        url_data.columns = ['index', 'date', 'urls']
        urls = list(url_data['urls'])

    # 데이터가 들어갈 공간
    data_set = {}



    #url 접속 후 크롤링
    print("### 본문 수집 ###")
    for idx in tqdm(range(len(urls))):

        # 초기화
        body = ''
        title = ''
        day = ''
        company = ''


        # 디지털타임스 외 타 신문사 제외
        if 'www.dt.co.kr' in urls[idx]:


            ### parsing ###
            try:
                req = requests.get(urls[idx], headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
                )
                t.sleep(0.3)
                page = req.content
                page = bs(page, 'html.parser')  

            except:
                continue
                
            body = page.find_all('div','art_txt')[0].text.lstrip().rstrip()
            title = page.title.text
            day = url_data['date'][idx]
            company = '디지털타임스'
            data_set[idx] = [day, title, body, company, urls[idx]]
                
            
        else:
            continue
            

    '''
    데이터 내보내기 (뉴스본문)
    '''
    dataframe = pd.DataFrame(data_set).transpose()
    if len(dataframe) ==0:
        print('자료가 없습니다.')
        pass
    else:
        dataframe.columns = ['date', 'title', 'body', 'company', 'url']
        # 본문 내 특수문자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[^\w\s\.\%\)\(\']', repl=r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(pat=r'["\xa0"]', repl = r'', regex=True)
        # 한자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u3400-\u9FBF]', repl = r'', regex=True)
        dataframe['body']= dataframe['body'].str.replace(pat=r'["\u2027","\u2026"]', repl = r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(' b ', '')
        dataframe = dataframe.reset_index()
    dataframe.to_csv('디지털타임스 크롤링.csv', encoding = "cp949", index= False)





################################################
end_time = t.time()
duration = round((end_time - start_time)/60, 2)


print('작업 완료!')
print('작업 소요시간 : 약 {}분'.format(duration))
print('기사 개수 : ', len(dataframe))
print('작업 완료 시간 : ', str(dt.datetime.now()).split(' ')[0], str(dt.datetime.now()).split(' ')[1].split('.')[0])

# 전자신문

In [ ]:
start_time  = t.time()

for is_data in range(1):
    
    # url 가져오기 (csv파일)
    url_data = pd.read_csv('전체기사 url주소.csv', encoding = 'cp949')
    
    if len(url_data)==0:
        print('기사가 없습니다.')
        continue
    else:
        url_data.columns = ['date', 'urls']

        # 특정 url만 필터링
        url_data = url_data[ url_data['urls'].str.contains('etnews')]

        url_data = url_data.reset_index()
        url_data.columns = ['index', 'date', 'urls']
        urls = list(url_data['urls'])

    # 데이터가 들어갈 공간
    data_set = {}



    #url 접속 후 크롤링
    print("### 본문 수집 ###")
    for idx in tqdm(range(len(urls))):

        # 초기화
        body = ''
        title = ''
        day = ''
        company = ''


        # 전자신문 외 타 신문사 제외
        if 'www.etnews.com' in urls[idx]:


            ### parsing ###
            try:
                req = requests.get(urls[idx], headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
                )
                t.sleep(0.3)
                page = req.content
                page = bs(page, 'html.parser')  

            except:
                continue
                
            body = page.body.p.text
            title = page.title.text
            day = url_data['date'][idx]
            company = '전자신문'
            data_set[idx] = [day, title, body, company, urls[idx]]
                
            
        else:
            continue
            

    '''
    데이터 내보내기 (뉴스본문)
    '''
    dataframe = pd.DataFrame(data_set).transpose()
    if len(dataframe) ==0:
        print('자료가 없습니다.')
        pass
    else:
        dataframe.columns = ['date', 'title', 'body', 'company', 'url']
        # 본문 내 특수문자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[^\w\s\.\%\)\(\']', repl=r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(pat=r'["\xa0"]', repl = r'', regex=True)
        # 한자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u3400-\u9FBF]', repl = r'', regex=True)
        dataframe['body']= dataframe['body'].str.replace(pat=r'["\u2027","\u2026"]', repl = r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(' b ', '')
        dataframe = dataframe.reset_index()
    dataframe.to_csv('전자신문 크롤링.csv', encoding = "cp949", index= False)





################################################
end_time = t.time()
duration = round((end_time - start_time)/60, 2)


print('작업 완료!')
print('작업 소요시간 : 약 {}분'.format(duration))
print('기사 개수 : ', len(dataframe))
print('작업 완료 시간 : ', str(dt.datetime.now()).split(' ')[0], str(dt.datetime.now()).split(' ')[1].split('.')[0])

# SBS 뉴스 ( news.sbs.co.kr )

In [94]:
start_time  = t.time()

for is_data in range(1):
    
    # url 가져오기 (csv파일)
    url_data = pd.read_csv('전체기사 url주소.csv', encoding = 'cp949')
    
    if len(url_data)==0:
        print('기사가 없습니다.')
        continue
    else:
        url_data.columns = ['date', 'urls']

        # 특정 url만 필터링
        url_data = url_data[ url_data['urls'].str.contains('news.sbs.co.kr')]

        url_data = url_data.reset_index()
        url_data.columns = ['index', 'date', 'urls']
        urls = list(url_data['urls'])

    # 데이터가 들어갈 공간
    data_set = {}



    #url 접속 후 크롤링
    print("### 본문 수집 ###")
    for idx in tqdm(range(len(urls))):

        # 초기화
        body = ''
        title = ''
        day = ''
        company = ''


        # 전자신문 외 타 신문사 제외
        if 'news.sbs.co.kr' in urls[idx]:


            ### parsing ###
            try:
                req = requests.get(urls[idx], headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
                )
                t.sleep(0.3)
                page = req.content
                page = bs(page, 'html.parser')  

            except:
                continue
            # body
            target = page.find_all('div', 'text_area')
            temp =  ''
            for line in target:
                temp = temp + line.text
            body = temp
            
            title = page.title.text
            day = url_data['date'][idx]
            company = 'sbs뉴스'
            data_set[idx] = [day, title, body, company, urls[idx]]
                
            
        else:
            continue
            
            


    '''
    데이터 내보내기 (뉴스본문)
    '''
    dataframe = pd.DataFrame(data_set).transpose()
    if len(dataframe) ==0:
        print('자료가 없습니다.')
        pass
    else:
        dataframe.columns = ['date', 'title', 'body', 'company', 'url']
        # 본문 내 특수문자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[^\w\s\.\%\)\(\']', repl=r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(pat=r'["\xa0"]', repl = r'', regex=True)
        # 한자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u3400-\u9FBF]', repl = r'', regex=True)
        dataframe['body']= dataframe['body'].str.replace(pat=r'["\u2027","\u2026"]', repl = r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(' b ', '')
        dataframe = dataframe.reset_index()
    dataframe.to_csv('sbs뉴스 크롤링.csv', encoding = "cp949", index= False)





################################################
end_time = t.time()
duration = round((end_time - start_time)/60, 2)


print('작업 완료!')
print('작업 소요시간 : 약 {}분'.format(duration))
print('기사 개수 : ', len(dataframe))
print('작업 완료 시간 : ', str(dt.datetime.now()).split(' ')[0], str(dt.datetime.now()).split(' ')[1].split('.')[0])

### 본문 수집 ###


  0%|          | 0/4 [00:00<?, ?it/s]

작업 완료!
작업 소요시간 : 약 0.04분
기사 개수 :  4
작업 완료 시간 :  2021-09-29 14:15:10


# SBS 뉴스 ( biz.sbs.co.kr ) - selenium

In [3]:
start_time  = t.time()


# webdriver
driver = webdriver.Chrome("./chromedriver", options = options)


for is_data in range(1):
    
    # url 가져오기 (csv파일)
    url_data = pd.read_csv('전체기사 url주소.csv', encoding = 'cp949')
    

    if len(url_data)==0:
        print('기사가 없습니다.')
        continue
    else:
        url_data.columns = ['date', 'urls']

        # 특정 url만 필터링
        url_data = url_data[ url_data['urls'].str.contains('biz.sbs.co.kr|cnbc.sbs.co.kr')]

        url_data = url_data.reset_index()
        url_data.columns = ['index', 'date', 'urls']
        urls = list(url_data['urls'])
        
    # 데이터가 들어갈 공간
    data_set = {}
    #url 접속 후 크롤링
    print("### 본문 수집 ###")
    
    
    for idx in tqdm(range(len(urls))):

        # 초기화
        body = ''
        title = ''
        day = ''
        company = ''


        # 전자신문 외 타 신문사 제외
        if ('biz.sbs.co.kr' in urls[idx]) or ('cnbc.sbs.co.kr' in urls[idx]):
            urls[idx] = urls[idx].replace('?division=NAVER', '')
            if 'cnbc.sbs.co.kr/read.jsp?pmArticleId' in urls[idx]:
                urls[idx] = 'https://biz.sbs.co.kr/article/' + urls[idx].split('ArticleId=')[-1]
            elif ('biz.sbs.co.kr' in urls[idx]) or ('cnbc.sbs.co.kr/article_hub' in urls[idx]):  
                urls[idx] = 'https://biz.sbs.co.kr/article/' + urls[idx].split('/')[-1]
                pass
            
            ### parsing ###
            driver.get(urls[idx])
            
            body = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#cnbc-front-articleContent-area-font'))).text
            title = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#cnbc-front-articleHeader-self > div > div > h3'))).text
            day = url_data['date'][idx]
            company = 'sbs뉴스'
            
            
            # 주식기사 제외
            if '[관심주]' in title:
                continue
            
            if '[주식상담]' in title:
                continue
                
                
            
            data_set[idx] = [day, title, body, company, urls[idx]]
                
            
        else:
            continue
            
            


    '''
    데이터 내보내기 (뉴스본문)
    '''
    dataframe = pd.DataFrame(data_set).transpose()
    if len(dataframe) ==0:
        print('자료가 없습니다.')
        pass
    else:
        dataframe.columns = ['date', 'title', 'body', 'company', 'url']
        # 본문 내 특수문자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[^\w\s\.\%\)\(\']', repl=r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(pat=r'["\xa0"]', repl = r'', regex=True)
        # 한자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u3400-\u9FBF]', repl = r'', regex=True)
        dataframe['body']= dataframe['body'].str.replace(pat=r'["\u2027","\u2026"]', repl = r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(' b ', '')
        dataframe = dataframe.reset_index()
        
    dataframe = dataframe.drop_duplicates(subset = ['body'])
    dataframe.to_csv('sbs-biz 크롤링.csv', encoding = "cp949", index= False)

driver.close()



################################################
end_time = t.time()
duration = round((end_time - start_time)/60, 2)


print('작업 완료!')
print('작업 소요시간 : 약 {}분'.format(duration))
print('기사 개수 : ', len(dataframe))
print('작업 완료 시간 : ', str(dt.datetime.now()).split(' ')[0], str(dt.datetime.now()).split(' ')[1].split('.')[0])

### 본문 수집 ###


  0%|          | 0/35 [00:00<?, ?it/s]

KeyboardInterrupt: 

# KBS

In [ ]:
start_time  = t.time()

for is_data in range(1):
    
    # url 가져오기 (csv파일)
    url_data = pd.read_csv('전체기사 url주소.csv', encoding = 'cp949')
    
    if len(url_data)==0:
        print('기사가 없습니다.')
        continue
    else:
        url_data.columns = ['date', 'urls']

        # 특정 url만 필터링
        url_data = url_data[ url_data['urls'].str.contains('etnews')]

        url_data = url_data.reset_index()
        url_data.columns = ['index', 'date', 'urls']
        urls = list(url_data['urls'])

    # 데이터가 들어갈 공간
    data_set = {}



    #url 접속 후 크롤링
    print("### 본문 수집 ###")
    for idx in tqdm(range(len(urls))):

        # 초기화
        body = ''
        title = ''
        day = ''
        company = ''


        # KBS 외 타 신문사 제외
        if 'www.etnews.com' in urls[idx]:


            ### parsing ###
            try:
                req = requests.get(urls[idx], headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
                )
                t.sleep(0.3)
                page = req.content
                page = bs(page, 'html.parser')  

            except:
                continue
                
            body = page.body.p.text
            title = page.title.text
            day = url_data['date'][idx]
            company = 'KBS'
            data_set[idx] = [day, title, body, company, urls[idx]]
                
            
        else:
            continue
            

    '''
    데이터 내보내기 (뉴스본문)
    '''
    dataframe = pd.DataFrame(data_set).transpose()
    if len(dataframe) ==0:
        print('자료가 없습니다.')
        pass
    else:
        dataframe.columns = ['date', 'title', 'body', 'company', 'url']
        # 본문 내 특수문자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[^\w\s\.\%\)\(\']', repl=r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(pat=r'["\xa0"]', repl = r'', regex=True)
        # 한자 제거
        dataframe['body'] = dataframe['body'].str.replace(pat=r'[\u3400-\u9FBF]', repl = r'', regex=True)
        dataframe['body']= dataframe['body'].str.replace(pat=r'["\u2027","\u2026"]', repl = r'', regex=True)
        dataframe['body'] = dataframe['body'].str.replace(' b ', '')
        dataframe = dataframe.reset_index()
    dataframe.to_csv('KBS 크롤링.csv', encoding = "cp949", index= False)





################################################
end_time = t.time()
duration = round((end_time - start_time)/60, 2)


print('작업 완료!')
print('작업 소요시간 : 약 {}분'.format(duration))
print('기사 개수 : ', len(dataframe))
print('작업 완료 시간 : ', str(dt.datetime.now()).split(' ')[0], str(dt.datetime.now()).split(' ')[1].split('.')[0])

In [4]:
url = 'https://biz.sbs.co.kr/article/10000710925'
req = requests.get(url, headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
)
t.sleep(0.3)
page = req.content
page = bs(page, 'html.parser')  
page

<!DOCTYPE html>

<html lang="ko">
<head>
<title>[관심주] 네오위즈·NHN엔터 '규제 리스크' 털고 훨훨 날까 - SBS Biz</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="SBS Biz, 뉴스, 증권, 경제, 글로벌경제, 산업, 금융, 산업, 부동산, 시사, 세계, 생활, 문화, 연예, 스포츠, 속보, 재테크, CEO, 교양, Biz, 라이브TV " name="keywords"/>
<meta content="월드 마켓 투데이 '국내 전략' - 김란 제이비스탁 본부장그리스 대통령 선출 실패에 대한 시장 관심은 미미하다고 볼 수 있겠다. 시장에서 어느 정도 예견됐던 이슈이기 때문에 글로벌 증시에 미치는 여파는 크지 않을 것이다. 미리 극단적인 해석을 할 필요는 없다.◇ 네오위즈게임즈·NHN엔터테인먼트 투자전략게임주의 주가 …" name="description"/>
<!-- google -->
<link href="https://biz.sbs.co.kr/article/10000710925" rel="canonical"/>
<script type="application/ld+json">
            {"@context":"http://schema.org","@id":"10000710925","@type":"NewsArticle","headline":"[관심주] 네오위즈·NHN엔터 '규제 리스크' 털고 훨훨 날까","datePublished":"2014-12-30T09:26:00+09:00","dateModified":"2014-12-30T09:26:00+09:00","description":"월드 마켓 투데이 &#39;국내 전략&#39; - 김란 제이비스탁 

In [116]:
elements = page.find_all('div', 'news_txt')
elements

[<div class="news_txt" itemprop="articleBody">
 <div class="news_img"><img alt='미 CDC "코로나 부스터샷 후 팔통증·피로 등 경미한 이상 반응"' onerror="ImUtil.SetErrorImage(this);" src="//image.imnews.imbc.com/news/2021/world/article/__icsFiles/afieldfile/2021/09/29/p20210928_16.jpg"/><p class="caption">[사진 제공: 연합뉴스]</p></div> 미국에서 코로나19 백신 추가접종, 즉 ' 부스터샷' 접종자가 신고한 이상 반응 대부분이 경미한 사안인 것으로 파악됐습니다.
 <br/> <br/>   미국 질병통제예방센터 CDC는 현지시간 28일 보고서를 통해 지난달 12일부터 이달 19일까지 2차 접종을 마치고 추가로 백신을 맞은 부스터샷 접종자 중 자발적으로 이상 반응을 신고한 2만2천191명을 대상으로 한 분석 자료를 공개하며 이같이 밝혔습니다.
 <br/> <br/> 2만2천191명 가운데 71%가 접종 부위 통증을, 56%가 피로를, 43%가 두통을 겪었다고 신고했습니다.
 <br/> <br/>  접종 이후 입원한 인원은 13명으로 집계됐으며 입원에 이르게 된 사유는 해당 조사에서 확인되지 않았습니다.
 <br/> <br/>  얀센 백신으로 최초 접종을 진행했거나 얀센 백신으로 부스터샷을 접종한 200명 미만의 소수를 제외하고, 99%가량이 2차 접종을 시행했던 mRNA 백신으로 3차 접종을 받은 것으로 파악됐습니다.
 <br/> <br/>  이번 신고는 부스터샷 접종 당일부터 7일 사이에 이뤄졌습니다.
 <br/> <br/>  한편 이 중 건강 진단 설문을 완료한 1만2천591명 중 각각 국소, 전신 반응이 있었다는 비율은 79.4%, 74.1%로 2차 접종 후 반응 신고 비율과 큰 차이가 없는 것으로 나타났습니다.
 <br/> <br/>   이를 근거로 CDC

In [114]:
el

<div class="news_txt" itemprop="articleBody">
<br/>
</div>
